In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

NameError: name 'ids' is not defined

In [3]:
results

{}

In [4]:
print("Hello World :)")

Hello World :)


In [5]:
get_ipython().system('pip install db-dtypes')

In [6]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# Constants
PRED_WEEK = 40

In [8]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [9]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [10]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [11]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [12]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [13]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [14]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [15]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [16]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [17]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [18]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [19]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [20]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [21]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [22]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [23]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [25]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [26]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,82b9ea58ff3d04a35df8deba80644a0e,1,18141,11802,16.0,31-33,23,61,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,18141,11802,26.0,72,23,31-33,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,18141,11802,18.0,23,31-33,56,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,18141,11802,20.0,23,23,56,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,18141,11802,17.0,23,31-33,48-49,True,12079.5


In [27]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [28]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [29]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [30]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,82b9ea58ff3d04a35df8deba80644a0e,1,18141,11802,16.0,31-33,23,61,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,18141,11802,26.0,72,23,31-33,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,18141,11802,18.0,23,31-33,56,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,18141,11802,20.0,23,23,56,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,18141,11802,17.0,23,31-33,48-49,True,12079.5


In [31]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [32]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [33]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [34]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,82b9ea58ff3d04a35df8deba80644a0e,1,0,0,16.0,0,0,0,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,0,0,26.0,1,0,1,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,0,0,18.0,2,1,2,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,0,0,20.0,2,0,2,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,0,0,17.0,2,1,3,True,12079.5


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [36]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,82b9ea58ff3d04a35df8deba80644a0e,1,0,0,16.0,0,0,0,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,0,0,26.0,1,0,1,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,0,0,18.0,2,1,2,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,0,0,20.0,2,0,2,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,0,0,17.0,2,1,3,True,12079.5
5,82b9ea58ff3d04a35df8deba80644a0e,6,0,0,17.0,2,2,1,True,12079.5
6,82b9ea58ff3d04a35df8deba80644a0e,7,0,0,12.0,2,3,3,True,12079.5
7,82b9ea58ff3d04a35df8deba80644a0e,8,0,0,25.0,1,0,2,True,12079.5
8,82b9ea58ff3d04a35df8deba80644a0e,9,0,0,13.0,2,3,4,True,12079.5
9,82b9ea58ff3d04a35df8deba80644a0e,10,0,0,14.0,2,3,0,True,12079.5


In [37]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.106604
tract           0.019753
total_claims    1.000000
emp1            0.066523
emp2            0.002203
emp3            0.067199
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [38]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [40]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [41]:
results

{'82b9ea58ff3d04a35df8deba80644a0e': [[18.0,
   41.0,
   21.0,
   12.0,
   17.0,
   14.0,
   23.0,
   17.266666666666666,
   17.266666666666666],
  [20.896880345341458,
   9.130473506516022,
   22.87353058101263,
   19.39728729897786,
   22.046340098497073,
   18.09687953663135,
   17.13547353944081,
   18.250118189720993,
   18.093050973346273]],
 'f0fdfe3859cd89a7d30ff7f15e2f06df': [[37.0,
   47.0,
   59.0,
   26.0,
   24.0,
   44.0,
   92.0,
   25.942857142857143,
   25.942857142857143],
  [21.44765836382559,
   22.209692321068832,
   24.069283763292503,
   28.756993975506106,
   25.95272874259826,
   19.14968641061973,
   21.65298402328811,
   23.325067309810358,
   23.426389967469525]],
 '3eaeeb015822cc1c6b77c914fae0d087': [[14.0,
   15.0,
   41.0,
   37.0,
   16.0,
   18.0,
   15.0,
   20.97142857142857,
   20.97142857142857],
  [18.501196144265123,
   13.787495819851756,
   18.30246031963,
   25.126096793410312,
   21.08096250600282,
   21.300256367981444,
   19.82575766928494,


In [42]:
pd.DataFrame.from_dict(results)

,82b9ea58ff3d04a35df8deba80644a0e,f0fdfe3859cd89a7d30ff7f15e2f06df,3eaeeb015822cc1c6b77c914fae0d087,dfa86215568c869a2e94ce3e5fcf1661,d6a4947ec7c2a78bd32ba4e30f3bba53,f43fb9e90c5ecf879016b159aaa17fcb,10bd992d086c48a05cb0f1e3df1e20f1,24f871b664684c15caad90bc237ff40d,98e63567a2f4907c9cb2637247651530,14ceb8012cb1b912044297c8933cfcfc,...,d2fb0b2935cd696dc7681a74797e86b7,516a760c644c21ff41a8cd20b6fb73bf,99b418342eb86b3aba7061fb1b7584a8,a01d4790949ed9261a56c55a5eb2c239,7ab86f26bdf5e743b0f0adf08df559e3,4765514a954c2fc3eb7c1993662e60e9,65f24aac652de33abaf3f227b41e2641,36c489a05d6a987ff961fdc67ccadbc1,9200a2b83d869882f514fa76e6bcd05c,8c9d2aa90948679972a9382aadcc6001
0,"[18.0, 41.0, 21.0, 12.0, 17.0, 14.0, 23.0, 17....","[37.0, 47.0, 59.0, 26.0, 24.0, 44.0, 92.0, 25....","[14.0, 15.0, 41.0, 37.0, 16.0, 18.0, 15.0, 20....","[18.166666666666668, 18.166666666666668, 18.16...","[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[60.0, 15.0, 20.0, 16.0, 35.0, 18.0, 13.0, 25....","[15.0, 56.0, 12.0, 18.0, 11.0, 54.0, 56.0, 22....","[10.0, 16.0, 15.3, 15.3, 15.3, 13.0, 15.190476...","[20.0, 28.0, 55.0, 17.0, 13.0, 39.0, 79.0, 25....","[68.0, 16.0, 20.0, 16.0, 20.0, 23.0, 28.0, 25....",...,"[26.0, 10.0, 12.0, 62.0, 11.0, 13.0, 13.0, 24....","[39.0, 31.0, 34.0, 31.0, 30.0, 46.0, 34.0, 35....","[45.0, 28.26086956521739, 28.26086956521739, 2...","[16.0, 19.0, 17.0, 20.0, 24.0, 51.0, 22.0, 22....","[23.09090909090909, 23.09090909090909, 26.0, 1...","[15.0, 17.0, 18.0, 15.0, 13.0, 16.0, 11.0, 16....","[14.11111111111111, 14.11111111111111, 10.0, 1...","[27.0, 21.125, 14.0, 20.705882352941178, 20.70...","[13.0, 19.73076923076923, 11.0, 15.0, 17.0, 61...","[43.0, 11.0, 11.0, 14.0, 14.0, 36.0, 31.0, 18...."
1,"[20.896880345341458, 9.130473506516022, 22.873...","[21.44765836382559, 22.209692321068832, 24.069...","[18.501196144265123, 13.787495819851756, 18.30...","[20.16496673767172, 19.85408591465807, 19.6269...","[14.74797303241212, 14.59988843195606, 13.8497...","[25.64623854664305, 27.4110976700494, 20.39435...","[22.146959926748234, 20.553077158248435, 19.02...","[11.640254891244695, 17.747668352662004, 13.55...","[18.505167142414795, 19.125666791228525, 19.23...","[33.27060661278665, 30.79540859136614, 20.4204...",...,"[31.780940384837454, 22.973032955715855, 17.47...","[35.65940488286896, 35.96964951184475, 35.5473...","[97.70996901532635, 28.063756443560123, 28.079...","[25.298323080883534, 25.72027511993784, 18.617...","[23.73139911610633, 23.6224216647679, 19.45633...","[12.21407969482243, 12.714153975248337, 13.626...","[14.428359568119049, 14.410810665751342, 14.01...","[23.735387033300412, 21.58978705573827, 16.392...","[24.705798507930012, 18.013136924651917, 17.54...","[13.755939019220836, 18.15337502183193, 17.337..."


In [43]:
pd.DataFrame.from_dict(results, orient='index')

,0,1
82b9ea58ff3d04a35df8deba80644a0e,"[18.0, 41.0, 21.0, 12.0, 17.0, 14.0, 23.0, 17....","[20.896880345341458, 9.130473506516022, 22.873..."
f0fdfe3859cd89a7d30ff7f15e2f06df,"[37.0, 47.0, 59.0, 26.0, 24.0, 44.0, 92.0, 25....","[21.44765836382559, 22.209692321068832, 24.069..."
3eaeeb015822cc1c6b77c914fae0d087,"[14.0, 15.0, 41.0, 37.0, 16.0, 18.0, 15.0, 20....","[18.501196144265123, 13.787495819851756, 18.30..."
dfa86215568c869a2e94ce3e5fcf1661,"[18.166666666666668, 18.166666666666668, 18.16...","[20.16496673767172, 19.85408591465807, 19.6269..."
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[14.74797303241212, 14.59988843195606, 13.8497..."
...,...,...
4765514a954c2fc3eb7c1993662e60e9,"[15.0, 17.0, 18.0, 15.0, 13.0, 16.0, 11.0, 16....","[12.21407969482243, 12.714153975248337, 13.626..."
65f24aac652de33abaf3f227b41e2641,"[14.11111111111111, 14.11111111111111, 10.0, 1...","[14.428359568119049, 14.410810665751342, 14.01..."
36c489a05d6a987ff961fdc67ccadbc1,"[27.0, 21.125, 14.0, 20.705882352941178, 20.70...","[23.735387033300412, 21.58978705573827, 16.392..."
9200a2b83d869882f514fa76e6bcd05c,"[13.0, 19.73076923076923, 11.0, 15.0, 17.0, 61...","[24.705798507930012, 18.013136924651917, 17.54..."


In [44]:
pd.DataFrame.from_dict(results, orient='index').columns = ['actual', 'prediction']

In [45]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df

,actual,prediction
82b9ea58ff3d04a35df8deba80644a0e,"[18.0, 41.0, 21.0, 12.0, 17.0, 14.0, 23.0, 17....","[20.896880345341458, 9.130473506516022, 22.873..."
f0fdfe3859cd89a7d30ff7f15e2f06df,"[37.0, 47.0, 59.0, 26.0, 24.0, 44.0, 92.0, 25....","[21.44765836382559, 22.209692321068832, 24.069..."
3eaeeb015822cc1c6b77c914fae0d087,"[14.0, 15.0, 41.0, 37.0, 16.0, 18.0, 15.0, 20....","[18.501196144265123, 13.787495819851756, 18.30..."
dfa86215568c869a2e94ce3e5fcf1661,"[18.166666666666668, 18.166666666666668, 18.16...","[20.16496673767172, 19.85408591465807, 19.6269..."
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[14.74797303241212, 14.59988843195606, 13.8497..."
...,...,...
4765514a954c2fc3eb7c1993662e60e9,"[15.0, 17.0, 18.0, 15.0, 13.0, 16.0, 11.0, 16....","[12.21407969482243, 12.714153975248337, 13.626..."
65f24aac652de33abaf3f227b41e2641,"[14.11111111111111, 14.11111111111111, 10.0, 1...","[14.428359568119049, 14.410810665751342, 14.01..."
36c489a05d6a987ff961fdc67ccadbc1,"[27.0, 21.125, 14.0, 20.705882352941178, 20.70...","[23.735387033300412, 21.58978705573827, 16.392..."
9200a2b83d869882f514fa76e6bcd05c,"[13.0, 19.73076923076923, 11.0, 15.0, 17.0, 61...","[24.705798507930012, 18.013136924651917, 17.54..."


In [46]:
print("Hello World :)")

Hello World :)


In [47]:
get_ipython().system('pip install db-dtypes')

In [48]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [49]:
# Constants
PRED_WEEK = 40

In [50]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [51]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [52]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [53]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [54]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [55]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [56]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [57]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [58]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [59]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [60]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [61]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [62]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [63]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [64]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [66]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [67]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [68]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,82b9ea58ff3d04a35df8deba80644a0e,1,18141,11802,16.0,31-33,23,61,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,18141,11802,26.0,72,23,31-33,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,18141,11802,18.0,23,31-33,56,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,18141,11802,20.0,23,23,56,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,18141,11802,17.0,23,31-33,48-49,True,12079.5


In [69]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [70]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [71]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [72]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,82b9ea58ff3d04a35df8deba80644a0e,1,18141,11802,16.0,31-33,23,61,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,18141,11802,26.0,72,23,31-33,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,18141,11802,18.0,23,31-33,56,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,18141,11802,20.0,23,23,56,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,18141,11802,17.0,23,31-33,48-49,True,12079.5


In [73]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [74]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [75]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [76]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,82b9ea58ff3d04a35df8deba80644a0e,1,0,0,16.0,0,0,0,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,0,0,26.0,1,0,1,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,0,0,18.0,2,1,2,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,0,0,20.0,2,0,2,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,0,0,17.0,2,1,3,True,12079.5


In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [78]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,82b9ea58ff3d04a35df8deba80644a0e,1,0,0,16.0,0,0,0,True,12079.5
1,82b9ea58ff3d04a35df8deba80644a0e,2,0,0,26.0,1,0,1,True,12079.5
2,82b9ea58ff3d04a35df8deba80644a0e,3,0,0,18.0,2,1,2,True,12079.5
3,82b9ea58ff3d04a35df8deba80644a0e,4,0,0,20.0,2,0,2,False,12079.5
4,82b9ea58ff3d04a35df8deba80644a0e,5,0,0,17.0,2,1,3,True,12079.5
5,82b9ea58ff3d04a35df8deba80644a0e,6,0,0,17.0,2,2,1,True,12079.5
6,82b9ea58ff3d04a35df8deba80644a0e,7,0,0,12.0,2,3,3,True,12079.5
7,82b9ea58ff3d04a35df8deba80644a0e,8,0,0,25.0,1,0,2,True,12079.5
8,82b9ea58ff3d04a35df8deba80644a0e,9,0,0,13.0,2,3,4,True,12079.5
9,82b9ea58ff3d04a35df8deba80644a0e,10,0,0,14.0,2,3,0,True,12079.5


In [79]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.106604
tract           0.019753
total_claims    1.000000
emp1            0.066523
emp2            0.002203
emp3            0.067199
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [80]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [82]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [83]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df

,actual,prediction
82b9ea58ff3d04a35df8deba80644a0e,"[18.0, 41.0, 21.0, 12.0, 17.0, 14.0, 23.0, 17....","[20.896880345341458, 9.130473506516022, 22.873..."
f0fdfe3859cd89a7d30ff7f15e2f06df,"[37.0, 47.0, 59.0, 26.0, 24.0, 44.0, 92.0, 25....","[21.44765836382559, 22.209692321068832, 24.069..."
3eaeeb015822cc1c6b77c914fae0d087,"[14.0, 15.0, 41.0, 37.0, 16.0, 18.0, 15.0, 20....","[18.501196144265123, 13.787495819851756, 18.30..."
dfa86215568c869a2e94ce3e5fcf1661,"[18.166666666666668, 18.166666666666668, 18.16...","[20.16496673767172, 19.85408591465807, 19.6269..."
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[14.74797303241212, 14.59988843195606, 13.8497..."
...,...,...
4765514a954c2fc3eb7c1993662e60e9,"[15.0, 17.0, 18.0, 15.0, 13.0, 16.0, 11.0, 16....","[12.21407969482243, 12.714153975248337, 13.626..."
65f24aac652de33abaf3f227b41e2641,"[14.11111111111111, 14.11111111111111, 10.0, 1...","[14.428359568119049, 14.410810665751342, 14.01..."
36c489a05d6a987ff961fdc67ccadbc1,"[27.0, 21.125, 14.0, 20.705882352941178, 20.70...","[23.735387033300412, 21.58978705573827, 16.392..."
9200a2b83d869882f514fa76e6bcd05c,"[13.0, 19.73076923076923, 11.0, 15.0, 17.0, 61...","[24.705798507930012, 18.013136924651917, 17.54..."


In [84]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df.explode(['actual','prediction])

SyntaxError: EOL while scanning string literal (2748506027.py, line 3)

In [85]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df.explode(['actual','prediction'])

,actual,prediction
82b9ea58ff3d04a35df8deba80644a0e,18.0,20.89688
82b9ea58ff3d04a35df8deba80644a0e,41.0,9.130474
82b9ea58ff3d04a35df8deba80644a0e,21.0,22.873531
82b9ea58ff3d04a35df8deba80644a0e,12.0,19.397287
82b9ea58ff3d04a35df8deba80644a0e,17.0,22.04634
...,...,...
8c9d2aa90948679972a9382aadcc6001,14.0,17.248208
8c9d2aa90948679972a9382aadcc6001,36.0,19.486518
8c9d2aa90948679972a9382aadcc6001,31.0,19.461606
8c9d2aa90948679972a9382aadcc6001,18.375,21.240518


In [86]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
weeks

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [87]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
PRED_WEEK

40

In [88]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
PRED_WEEK-10

30

In [89]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
test_weeks

[31, 32, 33, 34, 35, 36, 37, 38, 39]

In [90]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-11:]
test_weeks

[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

In [91]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [92]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [93]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
results_df

,actual,prediction
82b9ea58ff3d04a35df8deba80644a0e,17.0,6.306562
82b9ea58ff3d04a35df8deba80644a0e,18.0,20.89688
82b9ea58ff3d04a35df8deba80644a0e,41.0,9.130474
82b9ea58ff3d04a35df8deba80644a0e,21.0,22.873531
82b9ea58ff3d04a35df8deba80644a0e,12.0,19.397287
...,...,...
8c9d2aa90948679972a9382aadcc6001,14.0,17.248208
8c9d2aa90948679972a9382aadcc6001,36.0,19.486518
8c9d2aa90948679972a9382aadcc6001,31.0,19.461606
8c9d2aa90948679972a9382aadcc6001,18.375,21.240518


In [94]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
#results_df['week'] = []
1:10

SyntaxError: illegal target for annotation (3944715655.py, line 5)

In [95]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
#results_df['week'] = []
[1:10]

SyntaxError: invalid syntax (2934792075.py, line 5)